**Resouces:**

Explanation on the resnet architecture: input_size/output_size/kernel/stride at each layer:
https://medium.com/@14prakash/understanding-and-implementing-architectures-of-resnet-and-resnext-for-state-of-the-art-image-cf51669e1624

Resnet50 Architecture:
https://www.kaggle.com/keras/resnet50

Simple way of unpacking resnetX for stripping out FC layers and such:
https://discuss.pytorch.org/t/resnet-pretrained-model-with-last-fc-layer-stripped-does-not-work/17951

Reason as to why we want to resize each image and their labels to 224 x 224:
https://stackoverflow.com/questions/43922308/what-input-image-size-is-correct-for-the-version-of-resnet-v2-in-tensorflow-slim

How to modify the FC layer of resnet:
https://discuss.pytorch.org/t/how-to-modify-the-final-fc-layer-based-on-the-torch-model/766/3

**Data preprocessing requirement:**

In order to define the heatmap loss as torch.nn.functional.cross_entropy(input, target, weight=None, size_average=True, ignore_index=-100, reduce=True):

We need to have the target/label take on the form of (N, H, W), where the output needs to have (N, 12, H, W).

For each label tensor (H, W):
Each entry represents which join a location contains: value 1-11 are joints 1 to 11, value 0 represents no-joints at this location.

In [1]:
transform = transforms.Compose([
    Scale(0.5),
    ToTensor(),
])

hand_train = HandDataset('dataset.csv', transform=transform, train=True)
N = len(hand_train)
loader_train = DataLoader(hand_train, batch_size=64,
            sampler=sampler.SubsetRandomSampler(range(int(N*0.8))))

hand_val = HandDataset('dataset.csv', transform=transform, train=True)
loader_val = DataLoader(hand_val, batch_size=64,
            sampler=sampler.SubsetRandomSampler(range(int(N*0.8), int(N*0.9))))

hand_test = HandDataset('dataset.csv', transform=transform, train=False)
loader_test = DataLoader(hand_test, batch_size=64,
                         sampler=sampler.SubsetRandomSampler(range(int(N*0.9),N)))

# TODO: read data and create loader=(X_train, y_train)
X_train = []
y_train = []

for img_path, joints2d, joints3d in raw_train_data:
    img = cv2.imread(img_path)
    height, width, channels = img.shape
    # scale all image to 224 x 224, and scale their corresponding joint positions too!
    h_scale = 224.0 / height
    w_scale = 224.0 / width
    # scale joint positions
    # initialize labels to be zero (no joint) for all spatial locations
    label = np.zeros((224, 224))
    # Process pos2d and add to y_train
    for idx, pos2d in enumerate(joints2d):
        pos2d[0] = pos2d[0] * w_scale
        pos2d[1] = pos2d[1] * h_scale
        label[int(pos2d[0]), int(pos2d[1])] = idx + 1
    
    X_train.append(img)
    y_train.append(label)

trainDataset = zip(X_train, y_train)    

NameError: name 'raw_train_data' is not defined

In [2]:
import torch
import torch.nn as nn
import numpy as np
import torchvision.models as models

# Use renet 34 for speed, later may use resnet50 as our base NN stucture for joints detection
resnet34 = models.resnet34(pretrained=True).cuda()
for param in resnet34.parameters():
    # don't change/update the pretrained model parameters, only change the final fc layer
    param.requires_grad = False

# Upsample using transpose convolution and unpooling
# output_padding = 1 is intended to recover proper size
transConv1 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=0, output_padding=1)
transConv2 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=0, output_padding=1)
transConv3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=0, output_padding=1)
# Note: the following layer should have been maxunpooling, kernel 3 x 3, stride 2
# torch.nn.MaxUnpool2d() requires modification of the torchvision resnetXX modules
# to have the maxpooling layer return indices of max values, try ConvTranspose2d() instead
# If this doesn't work, have to and implement custom version of resnet
transConv4 = nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=3, stride=2, padding=0, output_padding=1)
transConv5 = nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=7, stride=2, padding=0, output_padding=1) # (224 x 224)

# finally, the join prediction Convolutional Layer, filter size 3 x 3, 21 + 1 = 22 filters?
# The +1 (filter dimension 11) is used to denote that the location does not correspond to any joints
jointPrediction = nn.Conv2d(in_channels=64, out_channels=21, kernel_size=3, stride=1, padding=1)

# stick a bunch of ReLu non-linearity in the upsampling pipeline
'''
model = nn.Sequential(
    *list(resnet34.children())[:-1], 
    transConv1,
    nn.ReLU(),
    transConv2,
    nn.ReLU(),
    transConv3,
    nn.ReLU(),
    transConv4,
    nn.ReLU(),
    transConv5,
    nn.ReLU(),
    jointPrediction
)
'''

# use a toy model to make sure that the location map implementations are correct
model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
)

# model.cuda()
# .cuda() makes model run on GPU

# optimizer using adam
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

for epoch in range(20):
    for X_batch, y_batch in trainDataset:
        # y_pred is of size 224 x 224 x 63, and let's say the filter dimension is layed out in the following way:
        # dimension 0-20: j1_x, j2_x, j3_x,...j21_x, dimension 21-41: j1_y, j2_y, j3_y, ... 
        # Above is for conveniently using broadcasting with heatmap element-wise product
        # y_pred is the predicted location maps for all 21 joints
        y_pred = model(X_batch)
        
        heatmap = 
        
        # use heatmap loss defined in VNect
        loss = torch.sum(torch.pow(y_batch.heatmap * (y_pred - y_batch.locMap), 2))
        
        loss.backward()
        optimizer.step()
        # Clears the gradients of all optimized torch.Tensor s
        optimizer.zero_grad()

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to C:\Users\haoha/.torch\models\resnet34-333f7ec4.pth
100.0%


RuntimeError: torch.cuda.FloatTensor is not enabled.